In [1]:
import pandas as pd
import numpy as np
import cobra
from CBModellingFuncs import *
import time
import os
import threadpoolctl

#### Important - define number of processes
processes = 50
####


threadpoolctl.threadpool_limits(limits=processes)
os.makedirs("../results/sampling/thinning_test", exist_ok=True)
os.makedirs("../results/dataframes/sampling", exist_ok=True)
cobra_config = cobra.Configuration()
cobra_config.solver = "cplex"

In [2]:
model, m_warnings = cobra.io.validate_sbml_model("../results/iMT1026-NZ.xml")

model.solver = 'cplex'
model.tolerance = 1e-6


rgdict = {}
for g in model.groups:
    for r in g.members:
        
        rgdict[r.id] = g.id.title()

SBML errors in validation, check error log for details.
COBRA errors in validation, check error log for details.
COBRA warnings in validation, check error log for details.


In [3]:
DRinfo = pd.read_csv(f"../results/dataframes/cultivation_data/DerivedRetentInfo.csv", index_col=0, header=[0,1])
IRinfo = pd.read_csv(f"../results/dataframes/cultivation_data/InterpolatedRetentInfo.csv", index_col=0, header=[0,1])
FRinfo = pd.read_csv(f"../results/dataframes/cultivation_data/FittedRetentInfo.csv", index_col=0, header=[0,1])

NGAMseries = pd.read_csv('../results/dataframes/NGAMseries.csv',index_col=0,squeeze=True)

model_viab_cols = ["Growth Rate model 1/h", "qS model mmol/gh","qP model g/gh", "qCO2 viable model mmol/gh", "qO2 viable model mmol/gh", "qStorGlyc mmol/gh"]
viab_cols = ["Average Growth Rate 1/h", "qS viable mmol/gh","qP g/gh", "qCO2 viable mmol/gh", "qO2 viable mmol/gh", "qStorGlyc mmol/gh"] 

## Sampling

In [4]:
co2=True
o2=True

- Uncomment the below cell to execute and produce the sampled chains

In [5]:
# %%time
# run_time = []
# sampling_results = []

# equation_names = ["Consensus", "Derived", "Interpolated", "Fitted","ScaledConsensus"]
# datasets = [DRinfo, DRinfo, IRinfo, FRinfo, DRinfo]

# for tfactor in [1,10,100, 1000]:
#     for i, equation in enumerate(equation_names):
#         data = datasets[i].loc[:, model_viab_cols].copy()
#         if equation in ["Consensus", "ScaledConsensus"]:
#             data["qStorGlyc mmol/gh"] = 0
        
#         time_0 = data.iloc[0,:]
#         name = time_0.name
#         values = time_0
#         mu = values[0]
#         sd = values[1]
    
#         loop_start = time.time()


#         with model as temp_model:
            
#             set_biomass_objective(temp_model, equation, name)

#             temp_model.reactions.ATPM.bounds = (NGAMseries[f"{equation}"],1000)
           
#             solution = constrain_predict(temp_model, values, co2=co2)
           
#             pred_mu = solution.objective_value
            
#             if equation in ["Consensus", "ScaledConsensus"]:
#                 temp_model.reactions.get_by_id(equation).bounds = (pred_mu*0.95, pred_mu)
                
#             else:
#                 temp_model.reactions.get_by_id(f"{equation}Biomass{name}").bounds = (pred_mu*0.95, pred_mu)

#             flux_span = constrain_predict(temp_model, values, co2=co2, fluxva=True, processes=processes,fopt=0)
#             remove_blocked(temp_model, flux_span, solution)


#             chains = constrain_predict(temp_model, values, sampling=True, chains=4, thinning=tfactor, 
#                                        loopless=True, processes=processes, co2=co2, n_samples=1250)

#                  # ensure all have same number of samples, may have
#             chains = [df.iloc[:min([df.shape[0] for df in chains]), :] for df in chains]
#             print(f"{equation} chains with thinning factor of  {tfactor} all have length of {chains[0].shape[0]}") 

#             for j, chain in enumerate(chains):
#                 chain["Chain"] = j+1
#             chains = pd.concat(chains, ignore_index=True)
#             chains.to_pickle(f"../results/sampling/thinning_test/Test{equation}{name}T{tfactor}.pkl.gz", 
#                                 compression={'method': 'gzip', 'compresslevel': 6, 'mtime': 1})
            
#         loop_end = time.time()
#         run_time.append((equation, tfactor, (loop_end - loop_start)/60))
                         
# run_time = pd.DataFrame(run_time, columns=["Equation", "Thinning", "Time"])
# run_time.to_csv("../results/dataframes/sampling/Thinning_RunTime.csv")

2387 reactions before making consistent, 1425 after
Consensus chains with thinning factor of  1 all have length of 1250
2387 reactions before making consistent, 1444 after
Derived chains with thinning factor of  1 all have length of 1250
2387 reactions before making consistent, 1439 after
Interpolated chains with thinning factor of  1 all have length of 1250
2387 reactions before making consistent, 1444 after
Fitted chains with thinning factor of  1 all have length of 1250
2387 reactions before making consistent, 1447 after
ScaledConsensus chains with thinning factor of  1 all have length of 1250
2387 reactions before making consistent, 1430 after
Consensus chains with thinning factor of  10 all have length of 1250
2387 reactions before making consistent, 1438 after
Derived chains with thinning factor of  10 all have length of 1250
2387 reactions before making consistent, 1441 after
Interpolated chains with thinning factor of  10 all have length of 1250
2387 reactions before making con

In [6]:
%%time

data = DRinfo.loc[:,model_viab_cols]

time_0 = data.iloc[0,:]
name = time_0.name
values = time_0
mu = values[0]

all_stats = pd.DataFrame()
n_chains = 4

for tfactor in [1,10,100, 1000]:#, 10000]:
    
    con_chains = pd.read_pickle(f"../results/sampling/thinning_test/TestConsensus{name}T{tfactor}.pkl.gz")
    con_chains = [con_chains.set_index("Chain").loc[j+1].reset_index(drop=True) for j in range(n_chains)]
    
    scaled_con_chains = pd.read_pickle(f"../results/sampling/thinning_test/TestScaledConsensus{name}T{tfactor}.pkl.gz")
    scaled_con_chains = [scaled_con_chains.set_index("Chain").loc[j+1].reset_index(drop=True) for j in range(n_chains)]
    
    der_chains = pd.read_pickle(f"../results/sampling/thinning_test/TestDerived{name}T{tfactor}.pkl.gz")
    der_chains = [der_chains.set_index("Chain").loc[j+1].reset_index(drop=True) for j in range(n_chains)]
   
    int_chains = pd.read_pickle(f"../results/sampling/thinning_test/TestInterpolated{name}T{tfactor}.pkl.gz")
    int_chains = [int_chains.set_index("Chain").loc[j+1].reset_index(drop=True) for j in range(n_chains)]
    
    fit_chains = pd.read_pickle(f"../results/sampling/thinning_test/TestFitted{name}T{tfactor}.pkl.gz")
    fit_chains = [fit_chains.set_index("Chain").loc[j+1].reset_index(drop=True) for j in range(n_chains)]
    
    shared_columns = list(set(con_chains[0].columns).intersection(scaled_con_chains[0].columns, 
                                                                  der_chains[0].columns, 
                                                                  int_chains[0].columns, 
                                                                  fit_chains[0].columns))
    
    con_chains = [df.loc[:,shared_columns] for df in con_chains]
    scaled_con_chains = [df.loc[:,shared_columns] for df in scaled_con_chains]
    der_chains = [df.loc[:,shared_columns] for df in der_chains]
    int_chains = [df.loc[:,shared_columns] for df in int_chains]
    fit_chains = [df.loc[:,shared_columns] for df in fit_chains]
 
    all_df = []

    for tup in [("Consensus", con_chains),
                ("ScaledConsensus", scaled_con_chains),
                ("Derived", der_chains),
                ("Interpolated", int_chains),
                ("Fitted", fit_chains)]:
    
        biocomp, model_chains = tup
        grouped, rxns = extractchains(model_chains)
        
        for_pool = list(zip(grouped,rxns))
      
        pool = Pool(processes=processes)
        all_series = pool.starmap(calculateDiagnostics, for_pool)
        pool.close()
        df = pd.concat(all_series,axis=1).T
       
        df["Biomass Composition"] = biocomp
        all_df.append(df)
        
    all_df = pd.concat(all_df, ignore_index=True)
    all_df["Thinning Factor"] = tfactor
    
    all_stats = pd.concat([all_stats, all_df], ignore_index=True)
        
all_stats.set_index(["Thinning Factor", "Biomass Composition"], inplace=True)
all_stats["Rhat Fails"] = all_stats["Rhat"] > 1.01
all_stats["ESS-Bulk Fails"] = all_stats["ESS-Bulk"] < 400
n_grouped_chains = all_stats.groupby(["Thinning Factor", "Biomass Composition"]).count()["Rxn"]


# summary_stats = all_stats[["Geweke Fails","ESS Split Fails", "Rhat Fails", "ESS-Bulk Fails"]]
summary_stats = all_stats[["Geweke Fails","Rhat Fails", "ESS-Bulk Fails"]]
summary_stats = (summary_stats > 0).groupby(["Thinning Factor", "Biomass Composition"]).sum()
temp_df = (summary_stats.divide(n_grouped_chains, axis=0) * 100)
# temp_df.columns=["% Geweke Fails","% seperated-ESS Fails","% Rhat Fails","% bulk-ESS Fails"]
temp_df.columns=["% Geweke Fails","% Rhat Fails","% bulk-ESS Fails"]
summary_stats = pd.concat([summary_stats,temp_df],axis=1)
summary_stats.to_csv(f"../results/dataframes/sampling/Thinning_AllStats.csv")

CPU times: user 14.5 s, sys: 48.9 s, total: 1min 3s
Wall time: 1min 11s


In [7]:
summary_stats

Geweke Fails  Rhat Fails  ESS-Bulk Fails  \
Thinning Factor Biomass Composition                                             
1               Consensus                     828         460             745   
                Derived                       907         722             749   
                Fitted                        852         644             674   
                Interpolated                  948         456             706   
                ScaledConsensus               818         618             798   
10              Consensus                     875         321             599   
                Derived                       851         574             590   
                Fitted                        826         524             592   
                Interpolated                  868         545             573   
                ScaledConsensus               841         459             551   
100             Consensus                     443           2               2   
                Derived                       441           2               3   
                Fitted                        734           9              10   
                Interpolated                  739          48              22   
                ScaledConsensus               689           8               3   
1000            Consensus                     251           2               0   
                Derived                       255           2               0   
                Fitted                        188           2               0   
                Interpolated                  185           2               0   
                ScaledConsensus               448           6               0   

                                     % Geweke Fails  % Rhat Fails  \
Thinning Factor Biomass Composition                                 
1               Consensus                 93.559322     51.977401   
                Derived                   99.779978     79.427943   
                Fitted                    97.594502     73.768614   
                Interpolated              97.933884     47.107438   
                ScaledConsensus           89.791438     67.837541   
10              Consensus                 94.390507     34.627832   
                Derived                   93.825799     63.285557   
                Fitted                    88.531618     56.162915   
                Interpolated              94.347826     59.239130   
                ScaledConsensus           94.813980     51.747463   
100             Consensus                 47.891892      0.216216   
                Derived                   46.033403      0.208768   
                Fitted                    76.458333      0.937500   
                Interpolated              78.118393      5.073996   
                ScaledConsensus           73.768737      0.856531   
1000            Consensus                 27.673649      0.220507   
                Derived                   27.301927      0.214133   
                Fitted                    20.258621      0.215517   
                Interpolated              19.659936      0.212540   
                ScaledConsensus           47.608927      0.637620   

                                     % bulk-ESS Fails  
Thinning Factor Biomass Composition                    
1               Consensus                   84.180791  
                Derived                     82.398240  
                Fitted                      77.205040  
                Interpolated                72.933884  
                ScaledConsensus             87.596048  
10              Consensus                   64.617044  
                Derived                     65.049614  
                Fitted                      63.451233  
                Interpolated                62.282609  
                ScaledConsensus             62.119504  
100             Consensus                    0.216216  
       

In [8]:
failures = all_stats[all_stats.loc[:,["Rhat Fails", "ESS-Bulk Fails"]].any(axis=1)].copy()
roi = ["G6PDH2","PGI","PYK","MDHm","CSm","AKGMALtm","NADH2_u6cm","NADH2_u6mh","CYOR_u6m","CYOOm","ATPS3m","ATPM"]
failures = failures[failures["Rxn"].isin(roi)]

failures.to_csv(f"../results/dataframes/sampling/Thinning_FailuresInterestingReactions.csv")
failures

Rxn Geweke Fails      Rhat  \
Thinning Factor Biomass Composition                                      
1               Consensus                ATPS3m            4  1.039586   
                Consensus                  MDHm            4  1.025747   
                Consensus                 CYOOm            4   1.02158   
                Consensus                   CSm            1  1.055778   
                Consensus                   PYK            4  1.058106   
...                                         ...          ...       ...   
10              Fitted                 AKGMALtm            4  1.037851   
                Fitted               NADH2_u6cm            4   1.05468   
                Fitted                 CYOR_u6m            4    1.0457   
                Fitted                      PGI            4  1.021911   
100             Interpolated                PYK            4   1.01127   

                                       ESS-Bulk     ESS-Tail  Rhat Fails  \
Thinning Factor Biomass Composition                                        
1               Consensus            200.119507   221.170529        True   
                Consensus            218.626674   228.144431        True   
                Consensus            210.925856   242.117459        True   
                Consensus            140.745101    90.707419        True   
                Consensus            214.630936    63.790578        True   
...                                         ...          ...         ...   
10              Fitted               147.392423   227.831686        True   
                Fitted               191.344854   359.922183        True   
                Fitted               199.295779   313.572007        True   
                Fitted               284.687934    353.59276        True   
100             Interpolated         606.969378  1209.769999        True   

                                     ESS-Bulk Fails  
Thinning Factor Biomass Composition                  
1               Consensus                      True  
                Consensus                      True  
                Consensus                      True  
                Consensus                      True  
                Consensus                      True  
...                                             ...  
10              Fitted                         True  
                Fitted                         True  
                Fitted                         True  
                Fitted                         True  
100             Interpolated                  False  

[111 rows x 7 columns]

In [9]:
failures = all_stats[all_stats.loc[:,["Rhat Fails", "ESS-Bulk Fails"]].any(axis=1)].copy()
failures.loc[:,"Subsystem"] = failures["Rxn"].apply(lambda x:rgdict.get(x))

test = failures[(failures["Rhat Fails"] > 0)^(failures["ESS-Bulk Fails"] > 0)]
sum_fails_by_subsystem = test.groupby(["Thinning Factor", "Biomass Composition", "Subsystem"]).sum()[["Rhat Fails", "ESS-Bulk Fails"]]
sum_fails_by_subsystem = sum_fails_by_subsystem.groupby(["Thinning Factor", "Subsystem"]).sum().unstack([0])
sum_fails_by_subsystem = sum_fails_by_subsystem.fillna(0)


sum_fails_by_subsystem.to_csv(f"../results/dataframes/sampling/Thinning_FailuresSubsystem.csv")
sum_fails_by_subsystem

Rhat Fails                  \
Thinning Factor                                          1     10   100  1000   
Subsystem                                                                       
Alanine And Aspartate Metabolism                          1.0   0.0  1.0  0.0   
Amino Sugar And Nucleotide Sugar Metabolism               0.0   0.0  0.0  0.0   
Arginine And Proline Metabolism                           0.0   7.0  0.0  0.0   
Biomass Composition                                       0.0   0.0  0.0  0.0   
Carbohydrate Metabolism                                   0.0   0.0  0.0  0.0   
Citric Acid Cycle                                         1.0   4.0  3.0  0.0   
Cysteine Metabolism                                       0.0   2.0  0.0  0.0   
Exchange Reaction                                         0.0   0.0  0.0  4.0   
Fatty Acid Biosynthesis                                   2.0   6.0  1.0  0.0   
Fatty Acid Degradation                                   26.0  20.0  0.0  0.0   
Folate Metabolism                                         0.0  10.0  3.0  0.0   
Fructose And Mannose Metabolism                           0.0   0.0  0.0  0.0   
Galactose Metabolism                                      0.0   0.0  0.0  0.0   
Glutamate Metabolism                                      2.0   0.0  1.0  0.0   
Glutamine Metabolism                                      0.0   0.0  0.0  0.0   
Glycerolipid Metabolism                                   2.0   4.0  0.0  0.0   
Glycine And Serine Metabolism                             1.0   0.0  5.0  0.0   
Glycolysis/Gluconeogenesis                                0.0   0.0  8.0  0.0   
Glycosylation Pathway                                     0.0   0.0  0.0  0.0   
Glyoxylate And Dicarboxylate Metabolism                   4.0   6.0  0.0  0.0   
Histidine Metabolism                                      0.0   3.0  0.0  0.0   
Inositol Phosphate Metabolism                             0.0   0.0  0.0  0.0   
Methanol Metabolism                                       1.0   1.0  0.0  0.0   
Methionine Metabolism                                     1.0   6.0  0.0  0.0   
N-Glycan Metabolism                                       0.0   2.0  0.0  0.0   
Nad Biosynthesis                                          0.0   5.0  0.0  0.0   
Nucleotide Metabolism                                     6.0  11.0  4.0  0.0   
Other                                                     0.0   0.0  0.0  0.0   
Other Amino Acid Metabolism                               2.0   3.0  4.0  0.0   
Oxidative Phosphorylation                                 2.0   1.0  0.0  0.0   
Pantothenate And Coa Biosynthesis                         0.0   2.0  1.0  0.0   
Pentose Phosphate Pathway                                 0.0   0.0  0.0  0.0   
Phospholipid Metabolism                                   2.0  13.0  0.0  0.0   
Porphyrin And Chlorophyll Metabolism                      0.0   0.0  0.0  0.0   
Propanoate Metabolism                                     0.0   0.0  0.0  0.0   
Pyridoxine Metabolism                                     7.0   6.0  1.0  0.0   
Pyruvate Metabolism                                       7.0   7.0  3.0  0.0   
Quinone Biosynthesis                                      2.0   0.0  0.0  2.0   
Recombinant Protein Production                            0.0   0.0  0.0  0.0   
Riboflavin Metabolism                                     0.0   1.0  2.0  0.0   
Sphingolipid Metabolism                                   8.0  26.0  0.0  0.0   
Sterol Metabolism                                         0.0   4.0  0.0  1.0   
Thiamine Metabolism                                       8.0  16.0  0.0  0.0   
Threonine And Lysine Metabolism                           0.0   2.0  0.0  0.0   
Transport, Er                                             0.0   8.0  0.0  0.0   
Transport, Extracellular                                  0.0   1.0  0.0  4.0   
Transport, Golgi Apparatus                                0.0   2.0  0.0  0.0   
T

In [10]:
rhat_fails = failures[failures["Rhat Fails"] > 0].groupby(["Thinning Factor", "Biomass Composition", "Subsystem"]).sum()
rhat_fails = rhat_fails.groupby(["Thinning Factor", "Biomass Composition"])["Rhat Fails"].nlargest(5)
rhat_fails.index = rhat_fails.index.droplevel([0,1])
rhat_fails = rhat_fails.to_frame()

rhat_fails.to_csv(f"../results/dataframes/sampling/Thinning_FailuresRhat.csv")
rhat_fails

Rhat Fails
Thinning Factor Biomass Composition Subsystem                                             
1               Consensus           Fatty Acid Biosynthesis                             60
                                    Transport, Mitochondrial                            42
                                    Nucleotide Metabolism                               26
                                    Transport, Peroxisomal                              20
                                    Valine, Leucine, And Isoleucine Metabolism          19
...                                                                                    ...
1000            Interpolated        Exchange Reaction                                    1
                                    Transport, Extracellular                             1
                ScaledConsensus     Transport, Mitochondrial                             3
                                    Quinone Biosynthesis                                 2
                                    Sterol Metabolism                                    1

[79 rows x 1 columns]

In [11]:
essbulk_fails = failures[failures["ESS-Bulk Fails"] > 0].groupby(["Thinning Factor", "Biomass Composition", "Subsystem"]).sum()
if not essbulk_fails.empty:
    sum_essbulk = essbulk_fails.groupby(["Thinning Factor", "Biomass Composition"])["ESS-Bulk Fails"].nlargest(5)
    sum_essbulk.index = sum_essbulk.index.droplevel([0,1])
    sum_essbulk = sum_essbulk.to_frame()
    sum_essbulk.to_csv(f"../results/dataframes/sampling/Thinning_FailuresSumESS.csv")
    
else:
    
    sum_essbulk = "No ESS Bulk fails"

sum_essbulk

ESS-Bulk Fails
Thinning Factor Biomass Composition Subsystem                                       
1               Consensus           Fatty Acid Biosynthesis                       77
                                    Transport, Mitochondrial                      58
                                    Sphingolipid Metabolism                       56
                                    Fatty Acid Degradation                        54
                                    Nucleotide Metabolism                         47
...                                                                              ...
100             Interpolated        Fatty Acid Biosynthesis                        2
                                    Alanine And Aspartate Metabolism               1
                ScaledConsensus     Sterol Metabolism                              1
                                    Threonine And Lysine Metabolism                1
                                    Transport, Mitochondrial                       1

[68 rows x 1 columns]

In [12]:
geweke_fails = failures[failures["Geweke Fails"] > 0].groupby(["Thinning Factor", "Biomass Composition", "Subsystem"]).count()
geweke_fails = geweke_fails.groupby(["Thinning Factor", "Biomass Composition"])["Geweke Fails"].nlargest(5)
geweke_fails.index = geweke_fails.index.droplevel([0,1])
geweke_fails = geweke_fails.to_frame()


geweke_fails.to_csv(f"../results/dataframes/sampling/Thinning_FailuresGeweke.csv")
geweke_fails

Geweke Fails
Thinning Factor Biomass Composition Subsystem                             
1               Consensus           Fatty Acid Biosynthesis             75
                                    Transport, Mitochondrial            58
                                    Sphingolipid Metabolism             57
                                    Nucleotide Metabolism               46
                                    Sterol Metabolism                   33
...                                                                    ...
1000            Interpolated        Exchange Reaction                    1
                                    Transport, Extracellular             1
                ScaledConsensus     Transport, Mitochondrial             3
                                    Quinone Biosynthesis                 2
                                    Sterol Metabolism                    1

[79 rows x 1 columns]

### Additional diagnostic plots of a selection of reactions (used in later analysis)
Additional plots include Geweke plots, Rank plots, local, quantile and evolution ESS plots

In [13]:
%%time

from matplotlib.backends.backend_pdf import PdfPages

data = DRinfo.loc[:,model_viab_cols]

time_0 = data.iloc[0,:]
name = time_0.name
values = time_0
mu = values[0]

n_chains = 4
interesting_reactions = ["G6PDH2","PGI","PC","PYK","MDHm", 
                         "CSm", "AKGMALtm","NADH2_u6cm","NADH2_u6mh",
                         "SUCD2_u6m","CYOR_u6m","CYOOm","ATPS3m"]

for rxn in interesting_reactions:

    with PdfPages(f'../results/plots/SamplingDiagnostics{rxn}.pdf') as pdf:
        
        for equation in ["Consensus", "Derived", "Interpolated", "Fitted","ScaledConsensus"]:
            
            fig = plt.figure(figsize=(20, 18), constrained_layout=True)
            spec = fig.add_gridspec(nrows=4, ncols=5, height_ratios=[1,1,1,1], hspace=0.1, bottom=0.1)
            
            for i, tfactor in enumerate([1,10,100, 1000]):
                
                chains_df = pd.read_pickle(f"../results/sampling/thinning_test/Test{equation}{name}T{tfactor}.pkl.gz")
                chains_df = [chains_df.set_index("Chain").loc[j+1].reset_index(drop=True) for j in range(n_chains)]
                
                grouped, rxns = extractchains(chains_df)
                
                chains = grouped[rxns.index(rxn)]
                cmap = plt.cm.get_cmap('tab10', 10)
                color_list = [matplotlib.colors.rgb2hex(cmap(j)[:3]) for j in range(cmap.N)][:4]

                axes = [fig.add_subplot(spec[i, j]) for j in np.arange(5)]

                plottrace(chains,axes[0],color_list)
                axes[0].set_ylabel("Trace Plot",fontsize=15)
                
                plotrank(chains,axes[1],color_list)
                axes[1].set_ylabel("Rank Plot",fontsize=15)
                
                az.plot_ess(chains,ax=axes[2], relative=True, kind="local")
                az.plot_ess(chains,ax=axes[3], relative=True, kind="quantile")
                az.plot_ess(chains,ax=axes[4], relative=True, kind="evolution")
                
                [axes[i].set_ylim(0,1.4) for i in [2,3,4]]
                [axes[i].set_title("") for i in [1,2,3,4]]
                [ax.tick_params(labelsize=15) for ax in axes]

                axes[2].set_title(f"{rxn} - {biocomp} - Thinning factor {tfactor}",fontsize=20, pad=30)

            pdf.savefig(fig)
            plt.close()
                    
               

CPU times: user 3min 29s, sys: 12 s, total: 3min 41s
Wall time: 3min 42s
